<a href="https://colab.research.google.com/github/ZEmRX-0/NLP_Project/blob/main/NLP_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!wget -P /content/ https://github.com/ZEmRX-0/NLP_Project/raw/main/Train.xls
!pip install accelerate

In [ ]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
ROOT_DIR = '/content/'

df= pd.read_excel(ROOT_DIR + "Train.xls")

df= df.sample(frac=1.0, random_state=42)

df.head()

,PhraseId,Phrase,Sentiment
19756,19757,I’m stuckI can’t get professional help and I f...,3
2240,2241,"oh boy, where to start! i guess my biggest str...",0
20681,20682,Is it possible I'm living a nightmare/in hellI...,3
3096,3097,work! we are short handed and so all the work ...,0
7840,7841,I GOT GOVERNMENT JOB\nFRIENDS MEETING\nFRIENDS...,1


In [ ]:
value_to_name = {
    0: "Sadness",
    1: "Happiness",
    2: "Depression",
    3: "Suicide",
    4: "Neutral",
}

df["Sentiment_name"] = df["Sentiment"].map(value_to_name.get)


In [ ]:
df.head()

,PhraseId,Phrase,Sentiment,Sentiment_name
19756,19757,I’m stuckI can’t get professional help and I f...,3,Suicide
2240,2241,"oh boy, where to start! i guess my biggest str...",0,Sadness
20681,20682,Is it possible I'm living a nightmare/in hellI...,3,Suicide
3096,3097,work! we are short handed and so all the work ...,0,Sadness
7840,7841,I GOT GOVERNMENT JOB\nFRIENDS MEETING\nFRIENDS...,1,Happiness


In [ ]:
labels = df['Sentiment_name'].unique().tolist()
labels = [s.strip() for s in labels ]
labels

['Suicide', 'Sadness', 'Happiness', 'Depression', 'Neutral']

In [ ]:
for key, value in enumerate(labels):
    print(value)

Suicide
Sadness
Happiness
Depression
Neutral


In [ ]:
NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [ ]:
label2id

{'Suicide': 0, 'Sadness': 1, 'Happiness': 2, 'Depression': 3, 'Neutral': 4}

In [ ]:
id2label

{0: 'Suicide', 1: 'Sadness', 2: 'Happiness', 3: 'Depression', 4: 'Neutral'}

In [ ]:
df.head()

,PhraseId,Phrase,Sentiment,Sentiment_name
19756,19757,I’m stuckI can’t get professional help and I f...,3,Suicide
2240,2241,"oh boy, where to start! i guess my biggest str...",0,Sadness
20681,20682,Is it possible I'm living a nightmare/in hellI...,3,Suicide
3096,3097,work! we are short handed and so all the work ...,0,Sadness
7840,7841,I GOT GOVERNMENT JOB\nFRIENDS MEETING\nFRIENDS...,1,Happiness


In [ ]:
df["Sentiment"]=df.Sentiment_name.map(lambda x: label2id[str(x).strip()])

In [ ]:
df.head()

,PhraseId,Phrase,Sentiment,Sentiment_name
19756,19757,I’m stuckI can’t get professional help and I f...,0,Suicide
2240,2241,"oh boy, where to start! i guess my biggest str...",1,Sadness
20681,20682,Is it possible I'm living a nightmare/in hellI...,0,Suicide
3096,3097,work! we are short handed and so all the work ...,1,Sadness
7840,7841,I GOT GOVERNMENT JOB\nFRIENDS MEETING\nFRIENDS...,2,Happiness


In [ ]:
df.Sentiment_name.value_counts().plot(kind='bar', figsize=(5,5))

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased", max_length=512)

In [ ]:
model = BertForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)

## Splitting df


In [ ]:
SIZE= df.shape[0]

train_texts= list(df.Phrase[:SIZE//2])

val_texts=   list(df.Phrase[SIZE//2:(3*SIZE)//4 ])

test_texts=  list(df.Phrase[(3*SIZE)//4:])

train_labels= list(df.Sentiment[:SIZE//2])

val_labels=   list(df.Sentiment[SIZE//2:(3*SIZE)//4])

test_labels=  list(df.Sentiment[(3*SIZE)//4:])

In [ ]:
len(train_texts)


13424

In [ ]:
len(train_texts), len(val_texts), len(test_texts)

(13424, 6712, 6712)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)


In [ ]:
train_dataloader = DataLoader(train_encodings, train_labels)

val_dataloader = DataLoader(val_encodings, val_labels)

test_dataset = DataLoader(test_encodings, test_labels)

## Training with Trainer Class

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """
    Computes accuracy, F1, precision, and recall for a given set of predictions.

    Args:
        pred (obj): An object containing label_ids and predictions attributes.
            - label_ids (array-like): A 1D array of true class labels.
            - predictions (array-like): A 2D array where each row represents
              an observation, and each column represents the probability of
              that observation belonging to a certain class.

    Returns:
        dict: A dictionary containing the following metrics:
            - Accuracy (float): The proportion of correctly classified instances.
            - F1 (float): The macro F1 score, which is the harmonic mean of precision
              and recall. Macro averaging calculates the metric independently for
              each class and then takes the average.
            - Precision (float): The macro precision, which is the number of true
              positives divided by the sum of true positives and false positives.
            - Recall (float): The macro recall, which is the number of true positives
              divided by the sum of true positives and false negatives.
    """
    # Extract true labels from the input object
    labels = pred.label_ids

    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)

    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')

    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)

    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }


In [ ]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='/content/model',
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    fp16=True,
    load_best_model_at_end=True
)

In [ ]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
     # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics= compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

In [ ]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """

    # Get the index of the class with the highest probability
    # argmax() finds the index of the maximum value in the tensor along a specified dimension.
    # By default, if no dimension is specified, it returns the index of the maximum value in the flattened tensor.
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label
    # Since pred_label_idx is a tensor containing a single value (the predicted class index),
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label


In [ ]:
# Test with a an example text in english
text = "make me a sandwich."
# "Machine Learning itself is moving towards more and more automated"
predict(text)

## Save model for inference

In [ ]:
model_path = "sentemintal_anal-model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

## Re-Load saved model for inference

In [ ]:
model_path = "sentemintal_anal-model"


model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
nlp= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
nlp